In [1]:
import pandas as pd
import cv2
import os
import mediapipe as mp
import time
import matplotlib.pyplot as plt

In [2]:
import pickle
model_filename = 'Model/knn_model.pkl'
knn = pickle.load(open(model_filename, 'rb'))

In [3]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.6,
                    min_tracking_confidence=0.6,
                    static_image_mode=False,
                    smooth_landmarks=True,
                    model_complexity=2)

In [4]:
video_name = 'WPushUp2.mp4'
video_path = 'Videos/' + video_name

In [5]:
frame_count = 0
frame_numbers = []
data_dict = {}
desired_width = 480
desired_height = 360

In [6]:
cap = cv2.VideoCapture(video_path)

In [7]:
# Assuming df is your DataFrame
def calculate_angle(row):
    import math
    row = abs(row)

    # Accessing columns by their index positions (0 to 5) from the row
    radians = math.atan2(row[5] - row[3], row[4] - row[2]) - math.atan2(row[1] - row[3], row[0] - row[2])
    angle = math.degrees(radians)
    if angle < 0:
        angle += 360
    return angle

In [8]:
while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    # Increment frame_count
    frame_count += 1
    print(f'Processing frame {frame_count}')

    # Resize the frame to the desired width and height
    image = cv2.resize(image, (desired_width, desired_height))

    # Convert the image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = pose.process(image_rgb)

    if results.pose_landmarks:

        frame_numbers.append(frame_count)
        data_dict = {f"{landmark.name}_Point_x": [] for landmark in mp_pose.PoseLandmark}
        data_dict.update({f"{landmark.name}_Point_y": [] for landmark in mp_pose.PoseLandmark})
        data_dict.update({f"{landmark.name}_Point_z": [] for landmark in mp_pose.PoseLandmark})
        data_dict.update({f"{landmark.name}_visibility": [] for landmark in mp_pose.PoseLandmark})

        for landmark in mp_pose.PoseLandmark:
            x_list_name = f"{landmark.name}_Point_x"
            y_list_name = f"{landmark.name}_Point_y"
            z_list_name = f"{landmark.name}_Point_z"
            visibility_list_name = f"{landmark.name}_visibility"

            data_dict[x_list_name].append(results.pose_landmarks.landmark[landmark].x)
            data_dict[y_list_name].append(results.pose_landmarks.landmark[landmark].y)
            data_dict[z_list_name].append(results.pose_landmarks.landmark[landmark].z)
            data_dict[visibility_list_name].append(results.pose_landmarks.landmark[landmark].visibility)
                
        df = pd.DataFrame(data_dict)
        Angles_df = pd.DataFrame()
            
        Angle_At_Right_Elbow = df[[
            'RIGHT_WRIST_Point_x', 'RIGHT_WRIST_Point_y',
            'RIGHT_ELBOW_Point_x', 'RIGHT_ELBOW_Point_y',
            'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y'
        ]]
        
        Angles_df['Angle_At_Right_Elbow'] = Angle_At_Right_Elbow.apply(calculate_angle, axis=1)
        
        Angle_At_Left_Elbow = df[[
            'LEFT_WRIST_Point_x', 'LEFT_WRIST_Point_y',
            'LEFT_ELBOW_Point_x', 'LEFT_ELBOW_Point_y',
            'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y'
        ]]
        Angles_df['Angle_At_Left_Elbow'] = Angle_At_Left_Elbow.apply(calculate_angle, axis=1)
        
        # Calculate angle at right shoulder
        Angle_At_Right_Shoulder = df[[
            'RIGHT_ELBOW_Point_x', 'RIGHT_ELBOW_Point_y',
            'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y',
            'RIGHT_HIP_Point_x', 'RIGHT_HIP_Point_y'
        ]]
        Angles_df['Angle_At_Right_Shoulder'] = Angle_At_Right_Shoulder.apply(calculate_angle, axis=1)
        
        # Calculate angle at left shoulder
        Angle_At_Left_Shoulder = df[[
            'LEFT_ELBOW_Point_x', 'LEFT_ELBOW_Point_y',
            'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y',
            'LEFT_HIP_Point_x', 'LEFT_HIP_Point_y'
        ]]
        Angles_df['Angle_At_Left_Shoulder'] = Angle_At_Left_Shoulder.apply(calculate_angle, axis=1)
        
        # Calculate angle at right knee (example)
        Angle_At_Right_Knee = df[[
            'RIGHT_HIP_Point_x', 'RIGHT_HIP_Point_y',
            'RIGHT_KNEE_Point_x', 'RIGHT_KNEE_Point_y',
            'RIGHT_ANKLE_Point_x', 'RIGHT_ANKLE_Point_y'
        ]]
        Angles_df['Angle_At_Right_Knee'] = Angle_At_Right_Knee.apply(calculate_angle, axis=1)
        
        # Calculate angle at left knee (example)
        Angle_At_Left_Knee = df[[
            'LEFT_HIP_Point_x', 'LEFT_HIP_Point_y',
            'LEFT_KNEE_Point_x', 'LEFT_KNEE_Point_y',
            'LEFT_ANKLE_Point_x', 'LEFT_ANKLE_Point_y'
        ]]
        Angles_df['Angle_At_Left_Knee'] = Angle_At_Left_Knee.apply(calculate_angle, axis=1)
        
        # Calculate angle at right hip
        Angle_At_Right_Hip = df[[
            'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y',
            'RIGHT_HIP_Point_x', 'RIGHT_HIP_Point_y',
            'RIGHT_KNEE_Point_x', 'RIGHT_KNEE_Point_y'
        ]]
        Angles_df['Angle_At_Right_Hip'] = Angle_At_Right_Hip.apply(calculate_angle, axis=1)
        
        # Calculate angle at left hip
        Angle_At_Left_Hip = df[[
            'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y',
            'LEFT_HIP_Point_x', 'LEFT_HIP_Point_y',
            'LEFT_KNEE_Point_x', 'LEFT_KNEE_Point_y'
        ]]
        Angles_df['Angle_At_Left_Hip'] = Angle_At_Left_Hip.apply(calculate_angle, axis=1)
        
        # Calculate angle at neck (example)
        Angle_At_Neck = df[[
            'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y',
            'NOSE_Point_x', 'NOSE_Point_y',
            'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y'
        ]]
        Angles_df['Angle_At_Neck'] = Angle_At_Neck.apply(calculate_angle, axis=1)
        
        prediction = knn.predict(Angles_df)
        
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
        
        cv2.putText(image_bgr, f"Prediction: {prediction}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        cv2.imshow('MediaPipe Pose', image_bgr)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

Processing frame 1


C:\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processing frame 2


C:\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processing frame 3


...

C:\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processing frame 1460


C:\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processing frame 1461
Processing frame 1462


KeyboardInterrupt: 